In [1]:
from transformers import pipeline

In [2]:
from bs4 import BeautifulSoup

In [3]:
import requests

In [4]:
summarizer = pipeline('summarization')

In [7]:
def summary(url, max_words, min_words):
    r = requests.get(url)
    soup = BeautifulSoup(r.text,'html.parser')
    results = soup.find_all(['h1','p'])
    text = [result.text for result in results]
    
    
    article = ' '.join(text)
    max_chunk = 500
    
    article = article.replace('.', '.<eos>')
    article = article.replace('?', '?<eos>')
    article = article.replace('!', '!<eos>')
    sentences = article.split('<eos')
    
    current_chunk = 0
    chunks = []

    for sentence in sentences:
        if len(chunks) == current_chunk+1:
            if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
                chunks[current_chunk].extend(sentence.split(' '))
            else:
                current_chunk += 1
                chunks.append(sentence.split(' '))
        else:
            chunks.append(sentence.split(' '))

    for chunk_id in range(len(chunks)):
        chunks[chunk_id] = ' '.join(chunks[chunk_id])

        
    res = summarizer(chunks, max_length=max_words, min_length=min_words, do_sample=False)
    final = ' '.join([summ['summary_text'] for summ in res])
    
    return final

In [10]:
url = "https://apnews.com/article/russia-ukraine-russia-london-europe-moscow-b158645ccf222e05aede08e26b9f62c1"

In [11]:
summary(url, 100, 20)

' Putin: Russia ready to discuss confidence-building measures . Putin: U.S. and NATO rejected Moscow’s demand to keep Ukraine and other ex-Soviet nations out of NATO, halt weapons deployments near Russian borders . Putin said Russia is ready to engage in talks on limiting the deployment of intermediate range missiles in Europe . The announcement buoyed world financial markets and the long-suffering ruble after weeks of escalation .  Russian Defense Ministry did not indicate where the withdrawing troops had been deployed or how many were leaving . It released images of tanks and armored vehicles rolling onto a train, and a tank commander saluting his forces . NATO Secretary-General Jens Stoltenberg said there are “some grounds for cautious optimism” for diplomatic efforts . Ukraine’s leaders expressed skepticism .  U.S. defense official: Russian ground units have been moving out of larger assembly areas for several days . Russian lawmakers urge Putin to recognize rebel-held areas in eas